# ION_SDK Setup

In [ ]:
#@title <---Run to install ION_SDK & Dependancies
%%capture
!pip install --upgrade git+https://bitbucket.org/freemens/ion_sdk.git@master



# Login to Edison

In [ ]:

from getpass import getpass
import ion_sdk.edison_api.edison_api as eapi
from ion_sdk.edison_api.models.factoryModel import EdisonGenericComponent, Model,CurrentState
apiKey=getpass(prompt='Enter your api Key: (go to https://edison.ionenergy.co/core/users/me/api-key) ', stream=None) 
edApi=eapi.Client(apiKey)

Enter your api Key: (go to https://edison.ionenergy.co/core/users/me/api-key) ··········


# Pushing CSV data onto an Asset
In this tutorial we will push data to a coffee machine virutal asset. 
- If you don't have an instance of the asset yet, [follow tutorial-1](https://https://colab.research.google.com/drive/1IKwz4Nd8lh51-nrNEd8b_nq8dJlCausr?usp=sharing).
- Download [this csv file](https://docs.google.com/spreadsheets/d/1ZpI5ygabudWF5zTGK276Yk-TLh1taYQb8nvOQlimInM/edit?usp=sharing) and add it to the temporary local space of the Collab notebook.




In [ ]:
import pandas as pd
import datetime

%load_ext google.colab.data_table
df=pd.read_csv("tutorial2-1.csv")
print(df.head(5))

df['date']=pd.to_datetime(df['date'])
df=df.set_index('date')

Serial_number = "CM01"
asset=edApi.getAsset(Serial_number)

uploadSensorList=edApi.refactorDataframeToAsset(df,asset)

asset.df=df
edApi.updateSensorDataByFile(asset,uploadSensorList)


The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
                  date  ON button state  Coffee Request state
0  02/18/2021 12:00:00              0.0                     0
1  02/18/2021 12:10:00              1.0                     0
2  02/18/2021 12:10:01              0.0                     0
3  02/18/2021 12:10:06              0.1                     1
4  02/18/2021 12:10:07              0.0                     0
('ON button state', 100)
('Coffee request button state', 95)
preparing payload
Payload sent successfully ! Checkout: https://edison.ionenergy.co/core/iot/connected_component/view/229883/graph


#Retreiving data

In [ ]:
import pandas as pd
Serial_number='CM01'
asset=edApi.getAsset(Serial_number)

sensorNameList=edApi.matchSensors(["ON button state","Coffee request button state"],asset)
req={
    "assets":[asset],
    "sensorNames":sensorNameList,
    "startDate":eapi.edisonDate(2020,2,18,7,00),
    "endDate":eapi.edisonDate(2020,2,18,15,00)
        }
edApi.getAssetDataFrame(**req)

pd.options.plotting.backend = "plotly"

fig = asset.df.plot()
fig.show()

NameError: ignored

# Generating new data

In [ ]:
class CoffeeMachine:
    def __init__(self):
        self.onButton=0
        self.onTimer=0
        self.onPeriod=600.0
        self.status=0

    def stateMachine(self,onButtonState,deltaT_s):
        self.onButton=onButtonState
        if self.onButton ==1:
            self.onTimer=self.onPeriod
        if self.onTimer>0:
            self.status=1
            self.onTimer=self.onTimer-deltaT_s
        else :
            self.status=0
        return self.status

In [ ]:
Serial_number='CM01'
asset=edApi.getAsset(Serial_number)

sensorNameList=edApi.matchSensors(["ON button state","Coffee request button state"],asset)
req={
    "assets":[asset],
    "sensorNames":sensorNameList,
    "startDate":eapi.edisonDate(2021,2,18,7,00),
    "endDate":eapi.edisonDate(2021,2,18,15,00)
        }
edApi.getAssetDataFrame(**req)
asset.df=asset.df.resample('1S').asfreq().ffill()
cm=CoffeeMachine()


asset.df['status'] = [cm.stateMachine(x,1) for x in asset.df['CM01ON button state']]
print(df.head(5))
uploadSensorList=edApi.refactorDataframeToAsset(asset.df,asset)

edApi.updateSensorDataByFile(asset,uploadSensorList)
  

getting sensors: ON button state-Coffee request button state from: 2021-02-18 07:00:00 to : 2021-02-18 15:00:00
[##############################] 100%                     CM01ON button state  CM01Coffee request button state
date                                                                     
2021-02-18 12:00:00                  0.0                                0
2021-02-18 12:10:00                  1.0                                0
2021-02-18 12:10:01                  0.0                                0
2021-02-18 12:10:06                  0.1                                1
2021-02-18 12:10:07                  0.0                                0
('Coffee request button state', 93)
('ON button state', 88)
WARNING Possible Sensor Name Mismatch !!!
('Status', 100)
preparing payload
Payload sent successfully ! Checkout: https://edison.ionenergy.co/core/iot/connected_component/view/229883/graph


# Using sensor to manage data

In [ ]:
Serial_number='CM01'
asset=edApi.getAsset(Serial_number)

sensorNameList=edApi.matchSensors(["ON button state","Coffee request button state"],asset)
req={
    "assets":[asset],
    "sensorNames":sensorNameList,
    "startDate":eapi.edisonDate(2021,2,18,7,00),
    "endDate":eapi.edisonDate(2021,2,18,15,00)
        }
edApi.getAssetDataFrame(**req)
asset.df=asset.df.resample('1S').asfreq().ffill()
cm=CoffeeMachine()

from ion_sdk.tools.sim import Sensor
from ion_sdk.tools.toolbox import plotSensors,dataframeFromSensors

sensors={}
sensors['Status']=Sensor(**{'data':[],'time':[],'lastSigVal':0.0, 'sigVarTh':0.001, 'unit':'#','axis':1,'synced':1,'plotted':1})
sensors['On button']=Sensor(**{'data':[],'time':[],'lastSigVal':0.0, 'sigVarTh':0.001, 'unit':'#','axis':1,'synced':0,'plotted':1})

[sensors['Status'].significantAppend(cm.stateMachine(x,1),y) for x,y in zip(asset.df['CM01ON button state'],asset.df.index)]
[sensors['On button'].significantAppend(x,y) for x,y in zip(asset.df['CM01ON button state'],asset.df.index)]
df=dataframeFromSensors(sensors)

print(df.head(5))
asset.df=df

fig=plotSensors(sensors,df)
fig.show()

uploadSensorList=edApi.refactorDataframeToAsset(asset.df,asset,sensors)
edApi.updateSensorDataByFile(asset,uploadSensorList)

getting sensors: ON button state-Coffee request button state from: 2021-02-18 07:00:00 to : 2021-02-18 15:00:00
[##############################] 100%                     On button  Status
date                                  
2021-02-18 12:10:00        1.0     1.0
2021-02-18 12:10:01        0.0     NaN
2021-02-18 12:10:06        0.1     NaN
2021-02-18 12:10:07        0.0     NaN
2021-02-18 12:20:00        NaN     0.0


('ON button state', 90)
WARNING Possible Sensor Name Mismatch !!!
('Status', 100)
preparing payload
Payload sent successfully ! Checkout: https://edison.ionenergy.co/core/iot/connected_component/view/229883/graph
